# Lesson 2: Interacting with a CSV Data

## Setup and connect to the Azure OpenAI endpoint

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [1]:
# load environment variables
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path='C:\MIConsulting\LangChain\langchain-academy\.env')

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GOOGLEAI_API_KEY = os.getenv("GOOGLEAI_API_KEY")

In [2]:
import os 
import platform
import pandas as pd

from IPython.display import Markdown, HTML, display
from langchain.schema import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    max_retries=3,
    api_key=GOOGLEAI_API_KEY,
)

## Load the dataset

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [3]:
os.makedirs("data",exist_ok=True)

if platform.system() == "Windows":
    # Use curl on Windows
    !curl -o {file_url} https://covidtracking.com/data/download/all-states-history.csv
else:
    # Use wget on Linux/Mac
    !wget https://covidtracking.com/data/download/all-states-history.csv -O {file_url}

file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  308k    0  308k    0     0   578k      0 --:--:-- --:--:-- --:--:--  580k
100 2674k    0 2674k    0     0  3578k      0 --:--:-- --:--:-- --:--:-- 3589k


## Prepare the Langchain dataframe agent

In [7]:
# from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm=model, df=df, verbose=True, allow_dangerous_code=True, max_iterations=20)

# agent.invoke("how many rows are there?")

## Design your prompt and ask your question

In [8]:
CSV_PROMPT_PREFIX = """
First set the pandas display options to show all the columns,
get the column names, then answer the question.
"""

CSV_PROMPT_SUFFIX = """
- **ALWAYS** before giving the Final Answer, try another method.
Then reflect on the answers of the two methods you did and ask yourself
if it answers correctly the original question.
If you are not sure, try another method.
- If the methods tried do not give the same result,reflect and
try again until you have two methods that have the same result.
- If you still cannot arrive to a consistent result, say that
you are not sure of the answer.
- If you are sure of the correct answer, create a beautiful
and thorough response using Markdown.
- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,
ONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.
- **ALWAYS**, as part of your "Final Answer", explain how you got
to the answer on a section that starts with: "\n\nExplanation:\n".
In the explanation, mention the column names that you used to get
to the final answer.
"""

QUESTION = """
How may patients were hospitalized during July 2020
in Texas, and nationwide as the total of all states?
Use the hospitalizedIncrease column
"""

agent.invoke(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX)



> Entering new AgentExecutor chain...
Thought: The user wants to know the number of hospitalized patients in Texas and nationwide during July 2020, using the `hospitalizedIncrease` column.
First, I need to set the pandas display options to show all columns.
Then, I need to get the column names to confirm the available columns.
After that, I will filter the dataframe for July 2020 and for the state 'TX'.
I will sum the `hospitalizedIncrease` for Texas.
Then, I will sum the `hospitalizedIncrease` for all states nationwide for July 2020.
Finally, I will present the answer and then try another method to verify the result.

Let's start by setting the display options and getting the column names.
Action: python_repl_ast
Action Input: import pandas as pd
pd.set_option('display.max_columns', None)
print(df.columns)Index(['date', 'state', 'death', 'deathConfirmed', 'deathIncrease',
       'deathProbable', 'hospitalized', 'hospitalizedCumulative',
       'hospitalizedCurrently', 'hospitalizedI

{'input': '\nFirst set the pandas display options to show all the columns,\nget the column names, then answer the question.\n\nHow may patients were hospitalized during July 2020\nin Texas, and nationwide as the total of all states?\nUse the hospitalizedIncrease column\n\n- **ALWAYS** before giving the Final Answer, try another method.\nThen reflect on the answers of the two methods you did and ask yourself\nif it answers correctly the original question.\nIf you are not sure, try another method.\n- If the methods tried do not give the same result,reflect and\ntry again until you have two methods that have the same result.\n- If you still cannot arrive to a consistent result, say that\nyou are not sure of the answer.\n- If you are sure of the correct answer, create a beautiful\nand thorough response using Markdown.\n- **DO NOT MAKE UP AN ANSWER OR USE PRIOR KNOWLEDGE,\nONLY USE THE RESULTS OF THE CALCULATIONS YOU HAVE DONE**.\n- **ALWAYS**, as part of your "Final Answer", explain how yo